# Maps

In [30]:
# Import dependencies
import requests
import json
import pandas as pd
import hvplot.pandas
from config import geoapify_key 
silence(FIXED_SIZING_MODE,True)

ValueError: Input to silence should be a warning object - not of type <class 'str'>

In [2]:
# Import Data and move index back
oakland_data = pd.read_csv("output_data/oakland_data.csv",index_col="datapoint")
kent_data    = pd.read_csv("output_data/kent_data.csv"   ,index_col="datapoint")
saginaw_data = pd.read_csv("output_data/saginaw_data.csv",index_col="datapoint")

oakland_data["Lat"] = 0.1
oakland_data["Lon"] = 0.1
saginaw_data["Lat"] = 0.1
saginaw_data["Lon"] = 0.1
kent_data["Lat"] = 0.1
kent_data["Lon"] = 0.1

In [3]:
# Set up function for gathering coordinates for zipcodes
start_url = "https://api.geoapify.com/v1/geocode/search?text="
end_url =  "&lang=en&limit=1&type=postcode&filter=countrycode:us&apiKey="+geoapify_key
def get_coords(df):
    lons = [0]
    lats = [0]
    zips = [0]
    for zipcode in df["Zipcode"].unique():
        response = requests.get(start_url+str(zipcode)+end_url).json()
        lon = response["features"][0]["properties"]["lon"]
        lat = response["features"][0]["properties"]["lat"]
        lons.append(lon)
        lats.append(lat)
        zips.append(zipcode)

    temp_df = pd.DataFrame({"Zipcode":zips,"Lon":lons,"Lat":lats})
    for i in df.index:
        zipcode = df.at[i,"Zipcode"]
        zip_loc = temp_df.loc[temp_df["Zipcode"]== zipcode]
        df.at[i,"Lon"] = zip_loc["Lon"]
        df.at[i,"Lat"] = zip_loc["Lat"]

In [4]:
# Run function for each dataframe to gather the coordinates
get_coords(oakland_data)
get_coords(kent_data)
get_coords(saginaw_data)

In [5]:
# Export to CSV to save the API
oakland_data.to_csv("output_data/oakland_latlon.csv",index_label = "datapoint")
saginaw_data.to_csv("output_data/saginaw_latlon.csv",index_label = "datapoint")
kent_data.to_csv("output_data/kent_latlon.csv",index_label = "datapoint")

In [6]:
# Import CSVs to continue using data
oakland_final = pd.read_csv("output_data/oakland_latlon.csv",index_col="datapoint")
kent_final = pd.read_csv("output_data/kent_latlon.csv"   ,index_col="datapoint")
saginaw_final = pd.read_csv("output_data/saginaw_latlon.csv",index_col="datapoint")

## Oakland County

In [31]:
def get_graphs(df,var,county,scale):
    for year in range(2011,2021):
        filtered_data = df.loc[df["Year"]==year]
        cur_map = filtered_data.hvplot.points("Lon","Lat",xlabel = "Longitude",ylabel="Latitude", 
                                              frame_width = 700, frame_height = 500, geo = True, tiles = "EsriStreet", 
                                              hover_cols = ["Zipcode",var], title = f"{county} County {var}: {year}", size = var, 
                                              scale = scale,color=var)
        hvplot.save(cur_map,f"output_data/map_plots/{county}_{var}_{year}",fmt="html")

In [32]:
# Total Population
get_graphs(oakland_final,"Total Population","Oakland",.2)

In [9]:
# Poverty Count
get_graphs(oakland_final,"Poverty Count","Oakland",.5)

In [10]:
# % Unemployed
get_graphs(oakland_final,"% Unemployed","Oakland",4)

In [11]:
# Adjusted Median Income
get_graphs(oakland_final,"Adjusted Median Income","Oakland",.1)

In [12]:
# Adjusted Median Rent
get_graphs(oakland_final,"Adjusted Median Rent","Oakland",1)

In [13]:
# Adjusted Median Home Value
get_graphs(oakland_final,"Adjusted Median Home Value","Oakland",.08)

## Kent County

In [14]:
# Total Population
get_graphs(kent_final,"Total Population","Kent",.2)

In [15]:
# Poverty Count
get_graphs(kent_final,"Poverty Count","Kent",.5)

In [16]:
# % Unemployed
get_graphs(kent_final,"% Unemployed","Kent",4)

In [17]:
# Adjusted Median Income
get_graphs(kent_final,"Adjusted Median Income","Kent",.1)

In [18]:
# Adjusted Median Rent
get_graphs(kent_final,"Adjusted Median Rent","Kent",1)

In [19]:
# Adjusted Median Home Value
get_graphs(kent_final,"Adjusted Median Home Value","Kent",.08)

## Saginaw County

In [20]:
# Total Population
get_graphs(saginaw_final,"Total Population","Saginaw",.2)

In [21]:
# Poverty Count
get_graphs(saginaw_final,"Poverty Count","Saginaw",.5)

In [22]:
# % Unemployed
get_graphs(saginaw_final,"% Unemployed","Saginaw",4)

In [23]:
# Adjusted Median Income
get_graphs(saginaw_final,"Adjusted Median Income","Saginaw",.1)

In [24]:
# Adjusted Median Rent
get_graphs(saginaw_final,"Adjusted Median Rent","Saginaw",1)

In [25]:
# Adjusted Median Home Value
get_graphs(saginaw_final,"Adjusted Median Home Value","Saginaw",.08)